In [1]:
import math

genesisTarget = 26959535291011309493156476344723991336010898738574164086137773096960
IS_EPSILON_TARGET_DEFINITION = True

class Block:
    # `target` and `_hash` are `int`s.
    def __init__(self, height, target, _hash, timestamp):
        self.height = height
        self.target = target
        self.timestamp = timestamp
        # As penultimate and most minimal difference targets differ only up to the third decimal.
        self.diff = int(round(genesisTarget / target, 4)) if IS_EPSILON_TARGET_DEFINITION else genesisTarget / target
        self.level = level(_hash, target)        

    def __hash__(self):
        return hash((self.height, self.target, self.diff, self.level))

    def __eq__(self, other):
        return isinstance(other, self.__class__) and self.__dict__ == other.__dict__

    def __lt__(self, other):
        return self.height < other.height

    def __repr__(self):
        return f"Block({self.height}, {self.target}, {self.diff}, {self.level})"

    def __str__(self):
        return f"height: {self.height}, target: {self.target}, diff: {self.diff}, level: {self.level}"


def level(_hash, target):
    ratio = _hash / target
    return math.floor(-math.log2(ratio))

In [10]:
proof = {
    0: [
        Block(1, 100, 12345, 1632218227),
        Block(2, 95, 12346, 1632218230),
        Block(3, 90, 12347, 1632218235),
        Block(4, 85, 12348, 1632218240),
        Block(5, 80, 12349, 1632218245),
        Block(6, 75, 12350, 1632218250),
        Block(7, 70, 12351, 1632218255),
        Block(8, 65, 12352, 1632218260),
        Block(9, 60, 12353, 1632218265),
        Block(10, 55, 12354, 1632218270),
    ],
    1: [
        Block(1, 100, 12355, 1632218228),
        Block(3, 90, 12357, 1632218236),
        Block(5, 80, 12359, 1632218246),
        Block(7, 70, 12361, 1632218256),
        Block(9, 60, 12363, 1632218266),
    ],
    2: [
        Block(1, 100, 12365, 1632218229),
        Block(5, 80, 12369, 1632218247),
        Block(9, 60, 12373, 1632218267),
    ],
}

proof_2 = {
    0: [
        Block(1, 100, 12345, 1632218227),
        Block(2, 95, 12346, 1632218230),
        Block(3, 90, 12347, 1632218235),
        Block(4, 85, 12348, 1632218240),
        Block(5, 80, 12349, 1632218245),
        Block(6, 75, 12350, 1632218250),
        Block(7, 70, 12351, 1632218255),
        Block(8, 65, 12352, 1632218260),
        Block(9, 60, 12353, 1632218265),
        Block(10, 55, 12354, 1632218270),
        Block(11, 50, 12355, 1632218275),
        Block(12, 45, 12356, 1632218280),
    ],
    1: [
        Block(1, 100, 12355, 1632218228),
        Block(3, 90, 12357, 1632218236),
        Block(5, 80, 12359, 1632218246),
        Block(7, 70, 12361, 1632218256),
        Block(9, 60, 12363, 1632218266),
        Block(11, 50, 12355, 1632218275),
        Block(12, 45, 12356, 1632218280),
    ],
    2: [
        Block(1, 100, 12365, 1632218229),
        Block(5, 80, 12369, 1632218247),
        Block(9, 60, 12373, 1632218267),
        Block(11, 50, 12355, 1632218275),
    ],
}

def print_proof(Π):
    for μ in sorted(Π.keys(), reverse=True):
        for block in Π[μ]:
            print(f"Block({block.height}) ", end="")
        if Π[μ]:
            print("")

In [11]:
def separate_into_π_and_χ(Π, k):
    π = {}
    χ = {}
    if not Π:
        return π, χ

    if len(Π[0]) <= k:
        for μ, blocks in Π.items():
            π[μ] = []
            χ[μ] = blocks
    else:
        π[0] = Π[0][:-k]
        χ[0] = Π[0][-k:]
        block_heights_to_remove = [block.height for block in χ[0]]
        for μ in range(1, len(Π)):
            χ[μ] = []
            stable_part_index = len(Π[μ])
            for block in reversed(Π[μ]):
                if block.height in block_heights_to_remove:
                    χ[μ].insert(0, block)
                    stable_part_index -= 1
                else:
                    break
            π[μ] = Π[μ][:stable_part_index]
    return π, χ

In [17]:
π, χ = separate_into_π_and_χ(proof, 5)
print_proof(π)

Block(1) Block(5) 
Block(1) Block(3) Block(5) 
Block(1) Block(2) Block(3) Block(4) Block(5) 


In [18]:
print_proof(χ)

Block(9) 
Block(7) Block(9) 
Block(6) Block(7) Block(8) Block(9) Block(10) 
